In [1]:
from src.models.Encodec import Encodec
encodec = Encodec()


/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [2]:
from audiocraft.modules.codebooks_patterns import DelayedPatternProvider

pattern_provider = DelayedPatternProvider(4)

import torch
test = torch.randn(1,24000*20)

test2 = {"wav" : torch.randn(2,1,24000*20),
        "original_lens" : torch.Tensor([[24000*13],[24000*15]]).int()}

codes = encodec(test)


In [3]:
from src.models.model import MuMRVQ
from src.models.Transformer.Encoder import VanillaEncoder
from src.models.Transformer.Decoder import VanillaDecoder
model = MuMRVQ(VanillaEncoder(layers=2), VanillaDecoder(layers=2) ,encodec, debug = False, pattern="none", mask_before=False, sequence_length=10)

In [4]:
out_ = model(test2)

tensor(975, dtype=torch.int32)
tensor(1125, dtype=torch.int32)
codes: torch.Size([2, 4, 1500])
padding_mask: torch.Size([2, 1501])
codes: torch.Size([2, 4, 10])
padding_mask: torch.Size([2, 10])
masking after with masking proba : 0.5
============== codes_masking ==============
tensor([[[ True, False, False, False,  True,  True,  True, False,  True, False],
         [ True, False, False, False,  True,  True,  True, False,  True, False],
         [ True, False, False, False,  True,  True,  True, False,  True, False],
         [ True, False, False, False,  True,  True,  True, False,  True, False]],

        [[False, False, False,  True, False,  True, False,  True,  True,  True],
         [False, False, False,  True, False,  True, False,  True,  True,  True],
         [False, False, False,  True, False,  True, False,  True,  True,  True],
         [False, False, False,  True, False,  True, False,  True,  True,  True]]])
torch.Size([2, 4, 10])
40 tokens were masked with random masking
shape

In [5]:
out_['codes']

tensor([[[1025,  732,  399,  732, 1025, 1025, 1025,  732, 1025,  732],
         [1025,  691,  691, 1008, 1025, 1025, 1025,  387, 1025,  691],
         [1025,  637,  954,   30, 1025, 1025, 1025,  155, 1025,  444],
         [1025,  199,  327,  165, 1025, 1025, 1025,  873, 1025,  751]],

        [[ 732,  732,  732, 1025,  732, 1025,  732, 1025, 1025, 1025],
         [ 387,  348, 1008, 1025,  574, 1025,  691, 1025, 1025, 1025],
         [ 900,  726,   53, 1025,  900, 1025,  851, 1025, 1025, 1025],
         [ 513,  751,  679, 1025,  272, 1025,  910, 1025, 1025, 1025]]])

In [6]:
from torch.nn import CrossEntropyLoss

CrossEntropyLoss(ignore_index=1026)(out_['logits'][:,:,:,1:],out_['codes'])

tensor(7.0937, grad_fn=<NllLoss2DBackward0>)

In [ ]:
dummy_path = 